In [3]:
# -*- coding utf-8 -*-
import numpy as np
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import pyflux as pf
import csv 
import matplotlib.pyplot as plt
from datetime import datetime
import xgboost as xgb
import os

splitDataPath = '../dataProcessing/splitData/splitData.csv'
testDataPath = '../dataProcessing/splitData/testData.csv'
splitLanePath = '../dataProcessing/splitByLaneData/'
testSplitLanePath = '../dataProcessing/testSplitByLaneData/'
testWeatherPath = '../dataSets/testing_phase1/weather (table 7)_test1.csv'
table3 = '../dataSets/training/links (table 3).csv'
table4 = '../dataSets/training/routes (table 4).csv'
table7 = '../dataProcessing/splitData/weather.csv'
arimaPicturePath = '../dataProcessing/arimaPicture/'
arimaPredict = '../dataProcessing/arimaPredict1/'

def getData(path):
    data = pd.read_csv(path)
    return data

def regularTime(s):
    time = datetime.strptime(s,"%Y-%m-%d %H:%M:%S")
    minute = time.minute
    time = time.replace(minute = (minute/20)*20,second = 0)
    return time.strftime("%Y-%m-%d %H:%M:%S")

def splitDataByLane(data,lanes,path):
#     label = ['intersection_id','tollgate_id','lane_id','starting_time','time_window','week','pressure','sea_pressure','wind_direction','wind_speed','temperature','rel_humidity','precipitation','travel_lane','travel_velocity','travel_length']
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        return
    label = ['lane_id','time','pressure','sea_pressure','wind_direction','wind_speed','temperature','rel_humidity','precipitation','travel_velocity']
    dataSet = data.copy()
    dataSet['travel_velocity'] = dataSet['travel_velocity'].apply(lambda x: np.array(x.split(';')).astype(float))
    dataSet['travel_lane'] = dataSet['travel_lane'].apply(lambda x: np.array(x.split(';')).astype(float))
    dataSet['travel_length'] = dataSet['travel_length'].apply(lambda x: np.array(x.split(';')).astype(float))
    dataSet['travel_seq'] = dataSet['travel_seq'].apply(lambda x: x.split(';'))
    dataSet['starting_time'] = dataSet['starting_time'].apply(regularTime)
    dataSet.fillna(-1)
    
    dataSet.drop(['vehicle_id','total_velocity','total_length'],axis = 1,inplace = True)
    
    
    weather = []
    weaTmp = dataSet[['starting_time','pressure','sea_pressure','wind_direction','wind_speed','temperature','rel_humidity','precipitation']]
    for name,group in weaTmp.groupby(['starting_time']):
        temp = []
        temp.extend(group.values[0].tolist())
        weather.append(temp)
    wLabel = ['time','pressure','sea_pressure','wind_direction','wind_speed','temperature','rel_humidity','precipitation']
    weather = pd.DataFrame(weather,columns = wLabel)
    
    print '==========================='
    print 'weather data is'
    print weather.info()
    
    result = pd.concat(list(dataSet['travel_seq'].apply(lambda x: pd.DataFrame(x,columns=['lane_id']))))
    result['time'] = result['lane_id'].apply(lambda x: x.split('#')[1])
    result['time'] = result['time'].apply(regularTime)
    result['lane_id'] = result['lane_id'].apply(lambda x: x.split('#')[0])
    result['travel_velocity'] = pd.concat(list(dataSet['travel_velocity'].apply(lambda x: pd.DataFrame(x))))
    result = result.groupby(['time','lane_id']).mean().reset_index()
    result = pd.merge(result,weather,how = 'left',on = ['time'])
    
    result['time'] = result['time'].apply(pd.to_datetime)
    
    print '==========================='
    print 'begain to split'
    print result.info()
    
    if not os.path.exists(path):
        os.makedirs(path)
    
    for lane in lanes:
        temp = result[:][result['lane_id'] == lane]
        temp.drop(['lane_id'],axis = 1).to_csv(path+'lane_'+lane+'.csv')
#       print '==========='
#       print lane
#       print dataByLink[lane].info()
    
    

def handleWeaInflu(data,isTest = False):
    label = ['travel_velocity']
    # some day (9-28,10-10) lack the information of weather
    handleData = data.copy()

    handleData = handleData['travel_velocity'][handleData['travel_velocity']<30].to_frame()
    
    # ga
    if not isTest:
        time=pd.DataFrame(pd.date_range(start = '20160719000000',end = '20161017234000',freq = '20min'),columns=['time'])
    else:
        temp1 = pd.DataFrame(pd.date_range(start = '20161018060000',end = '20161018074000',freq = '20min'),columns=['time'])
        temp2 = pd.DataFrame(pd.date_range(start = '20161018150000',end = '20161018164000',freq = '20min'),columns=['time'])
        times = []
        times.append(temp1.copy())
        times.append(temp2.copy())
        for i in range(6):
            temp1['time'] = temp1['time'] + pd.DateOffset(days = 1)
            temp2['time'] = temp2['time'] + pd.DateOffset(days = 1)
            times.append(temp1.copy())
            times.append(temp2.copy())
        
        time = pd.concat(times)
        time.reset_index()
        
        
    result = pd.merge(time,handleData,how='left',left_on='time',right_index=True)
    result.set_index('time',inplace = True)
    result = result.interpolate()
    result.fillna(method = 'pad')

    return result

# def recoverWea(data): 
    
# def getErrorXlf(data):
#     train = data.copy()
#     handleData = handleWeaInflu(data)
#     result = 
    
def predict():
    routes = {}
    links = {}
    lanes = []
    with open(table3,'r') as fr:
        lines = csv.reader(fr)
        for line in lines:
            if lines.line_num == 1:
                continue
            # length,lane
            links[line[0]] = [int(line[1]),int(line[3])]
            lanes.append(line[0])

    with open(table4,'r') as fr:
        lines = csv.reader(fr)
        for line in lines:
            if lines.line_num == 1:
                continue
            if line[0] not in routes.keys():
                routes[line[0]] = {}
            routes[line[0]][line[1]] = line[2].split(',')
            
            
    print '==========================='
    print 'preparation completes'
    
    trainData = getData(splitDataPath)
    testData = getData(testDataPath)
    splitDataByLane(trainData,lanes,splitLanePath)
    splitDataByLane(testData,lanes,testSplitLanePath)
    predictLaneVelo = {}
    
    
    print '==========================='
    print 'prepate with lane'
    if not os.path.exists(arimaPicturePath):
        os.makedirs(arimaPicturePath)
    if not os.path.exists(arimaPredict):
        os.makedirs(arimaPredict)
    
    for lane in lanes:
   
        print '==========================='
        print lane
        data = pd.read_csv(splitLanePath+'lane_'+lane+'.csv')
        data['time'] = data['time'].apply(pd.to_datetime)
        data.set_index('time',inplace = True)
        testData= pd.read_csv(testSplitLanePath+'lane_'+lane+'.csv')
        testData['time'] = testData['time'].apply(pd.to_datetime)
        testData.set_index('time',inplace = True)
        
#         xlf = getErrorXlf(data)
        testData = handleWeaInflu(testData,True)
        handleData = handleWeaInflu(data).dropna()
#         handleData.plot(figsize = (100,10))
        result = []
#         plt.savefig(arimaPicturePath+lane+'_trace_picture.png')
#         plot_acf(handleData).savefig(arimaPicturePath+lane+'_acf_picture.png')
#         plot_pacf(handleData).savefig(arimaPicturePath+lane+'_pacf_picture.png')

        i = 0
        size = 14
        l = 42
        maxar = 6
        maxma = 6
        while(i<size):

            if i%2!=0:
                print handleData.tail()
                predict = run_aram(handleData[-l:],maxar,maxma,12)
                handleData = pd.concat([handleData,predict])
                
                predict = run_aram(handleData[-l:],maxar,maxma,9)
                handleData = pd.concat([handleData,predict,testData[i*6:(i+1)*6]])
                
                result.append(predict[0:6])
                print handleData.tail()
                                        
                predict = run_aram(handleData[-l:],maxar,maxma,12)
                handleData = pd.concat([handleData,predict])
                predict = run_aram(handleData[-l:],maxar,maxma,9)
                handleData = pd.concat([handleData,predict])
                result.append(predict[0:6])
            else:
                print handleData.tail()
                predict = run_aram(handleData[-l:],maxar,maxma,9)
                handleData = pd.concat([handleData,predict])
                
                predict = run_aram(handleData[-l:],maxar,maxma,9)
                handleData = pd.concat([handleData,predict,testData[i*6:(i+1)*6]])
            
            i = i+1
        temp = pd.concat(result)
        temp.to_csv(arimaPredict+'lane_'+lane+'.csv')

        
    
    for lane in links.keys():
        predictLaneVelo[lane] = pd.read_csv(arimaPredict+'lane_'+lane+'.csv')
        # recover weather
        predictLaneVelo[lane]['travel_time'] = float(links[lane][0])/predictLaneVelo[lane]['travel_velocity']
    
    result = []
    for intersection in routes.keys():
        for tollgate in routes[intersection].keys():
            lane = routes[intersection][tollgate][0]
            temp = predictLaneVelo[lane].copy()
            temp['intersection'] = intersection
            temp['tollgate'] = tollgate
            temp.drop(['travel_time','travel_velocity'],axis = 1,inplace = True)
            temp['time_window'] = temp['time']
            temp['time_window'] = temp['time_window'].apply(lambda x: '['+str(x)+','+str(pd.to_datetime(x)+pd.DateOffset(minutes = 20))+')')
            temp.drop(['time'],axis =1 ,inplace = True)
            temp['avg_travel_time'] = 0
            for lane in routes[intersection][tollgate]:
                temp['avg_travel_time'] = temp['avg_travel_time']+predictLaneVelo[lane]['travel_time']
            result.append(temp)
    

    result = pd.concat(result)    
    result.to_csv(arimaPredict+'result.csv',index = False)
    
        

def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    print dftest
    return dftest[1]

def best_diff(df, maxdiff = 8):
    p_set = {}
    for i in range(0, maxdiff):
        temp = df.copy() #每次循环前，重置
        if i == 0:
            temp['diff'] = temp[temp.columns[0]]
        else:
            temp['diff'] = temp[temp.columns[0]].diff(i)
            temp = temp.drop(temp.iloc[:i].index) #差分后，前几行的数据会变成nan，所以删掉
        pvalue = test_stationarity(temp['diff'])
        p_set[i] = pvalue
        p_df = pd.DataFrame.from_dict(p_set, orient="index")
        p_df.columns = ['p_value']
    i = 0
    bestdiff = 0
    while i < len(p_df):
        if p_df['p_value'][i]<0.01:
            bestdiff = i
            break
        i += 1
    return bestdiff

def produce_diffed_timeseries(df, diffn):
    if diffn != 0:
        df['diff'] = df[df.columns[0]].apply(lambda x:float(x)).diff(diffn)
    else:
        df['diff'] = df[df.columns[0]].apply(lambda x:float(x))
    df.dropna(inplace=True) #差分之后的nan去掉
    return df

def choose_order(ts, maxar, maxma):
    print 'choose order'
    order = sm.tsa.arma_order_select_ic(ts, maxar, maxma, ic=['aic', 'bic', 'hqic'],trend='c', fit_kw=dict(method='css'))
    print 'finish'
    return order.bic_min_order

def predict_recover(ts, df, diffn):
    if diffn != 0:
        ts.iloc[0] = ts.iloc[0]+df[df.columns[0]][-diffn]
        ts = ts.cumsum()
#     ts = np.exp(ts)
#    ts.dropna(inplace=True)
    print('还原完成')
    return ts


def run_aram(df, maxar, maxma, test_size):
    
    train = df
#     train['log'] = np.log(train[train.columns[0]])
    diffn = 0
    if test_stationarity(train[train.columns[0]]) < 0.01:
        train['diff'] = df[df.columns[0]]
        print('平稳')
    else:
        diffn = best_diff(train, maxdiff = 8)
        train = produce_diffed_timeseries(train, diffn)
        print('差分阶数为'+str(diffn)+'，已完成差分')
    print('开始进行ARMA拟合')
    order = choose_order(train[train.columns[0]], maxar, maxma)
    print('模型的阶数为：'+str(order))
    _ar = order[0]
    _ma = order[1]
    model = pf.ARIMA(data=train, ar=_ar, ma=_ma, integ=diffn,target='diff',family=pf.Normal())
    x = model.fit("MLE")
    x.summary()
    test_predict = model.predict(test_size)
    test_predict.columns = ['travel_velocity']
    test_predict.index = test_predict.index + pd.DateOffset(minutes = test_size*20)
    test_predict = predict_recover(test_predict, train, diffn)
    print test_predict.head()
    return test_predict
    

    
    

    
    
    
    
    

In [ ]:
predict()

preparation completes
prepate with lane
100
                     travel_velocity
time                                
2016-10-17 22:20:00         8.686026
2016-10-17 22:40:00        13.242009
2016-10-17 23:00:00         9.430894
2016-10-17 23:20:00         5.852252
2016-10-17 23:40:00         5.852252
(-5.0003204249791988, 2.2160771264182068e-05, 0, 41, {'5%': -2.9351348158036012, '1%': -3.6009833671885199, '10%': -2.6059629803688282}, 126.83718605014606)
平稳
开始进行ARMA拟合
choose order
finish
模型的阶数为：(4, 3)
Normal ARIMA(4,0,3)                                                                                       
======================================================= ==================================================
Dependent Variable: diff                                Method: MLE                                       
Start Date: 2016-10-17 11:20:00                         Log Likelihood: -53.1519                          
End Date: 2016-10-17 23:40:00                           AIC: 124

finish
模型的阶数为：(6, 0)
Normal ARIMA(6,1,0)                                                                                       
======================================================= ==================================================
Dependent Variable: Differenced diff                    Method: MLE                                       
Start Date: 2016-10-18 00:20:00                         Log Likelihood: -53.4185                          
End Date: 2016-10-18 11:40:00                           AIC: 122.8369                                     
Number of observations: 34                              BIC: 135.0478                                     
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 0.012      0.2011     0.0595   0.9526   (-0.3821 | 0.4061)       
AR(1)           

finish
模型的阶数为：(6, 1)
Normal ARIMA(6,1,1)                                                                                       
======================================================= ==================================================
Dependent Variable: Differenced diff                    Method: MLE                                       
Start Date: 2016-10-18 12:20:00                         Log Likelihood: -86.8502                          
End Date: 2016-10-18 23:40:00                           AIC: 191.7004                                     
Number of observations: 34                              BIC: 205.4377                                     
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 -0.2899    0.0829     -3.4968  0.0005   (-0.4523 | -0.1274)      
AR(1)           

finish
模型的阶数为：(6, 2)
Normal ARIMA(6,0,2)                                                                                       
======================================================= ==================================================
Dependent Variable: diff                                Method: MLE                                       
Start Date: 2016-10-19 00:00:00                         Log Likelihood: -77.5839                          
End Date: 2016-10-19 11:40:00                           AIC: 175.1679                                     
Number of observations: 36                              BIC: 191.0031                                     
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 9.1622     3.1564     2.9027   0.0037   (2.9756 | 15.3489)       
AR(1)           

finish
模型的阶数为：(6, 3)
Normal ARIMA(6,1,3)                                                                                       
======================================================= ==================================================
Dependent Variable: Differenced diff                    Method: MLE                                       
Start Date: 2016-10-19 09:20:00                         Log Likelihood: -28.1968                          
End Date: 2016-10-19 20:40:00                           AIC: 78.3936                                      
Number of observations: 34                              BIC: 95.1836                                      
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 0.001      0.0734     0.0134   0.9893   (-0.143 | 0.1449)        
AR(1)           

...

finish
模型的阶数为：(6, 0)
Normal ARIMA(6,1,0)                                                                                       
======================================================= ==================================================
Dependent Variable: Differenced diff                    Method: MLE                                       
Start Date: 2016-10-23 09:20:00                         Log Likelihood: -79.2611                          
End Date: 2016-10-23 20:40:00                           AIC: 174.5222                                     
Number of observations: 34                              BIC: 186.7331                                     
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 0.0482     0.4278     0.1128   0.9102   (-0.7902 | 0.8867)       
AR(1)           

finish
模型的阶数为：(6, 2)
Normal ARIMA(6,0,2)                                                                                       
======================================================= ==================================================
Dependent Variable: diff                                Method: MLE                                       
Start Date: 2016-10-23 20:00:00                         Log Likelihood: -49.4366                          
End Date: 2016-10-24 07:40:00                           AIC: 118.8731                                     
Number of observations: 36                              BIC: 134.7083                                     
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 15.2042    19.825     0.7669   0.4431   (-23.6528 | 54.0612)     
AR(1)           

finish
模型的阶数为：(6, 1)
Normal ARIMA(6,1,1)                                                                                       
======================================================= ==================================================
Dependent Variable: Differenced diff                    Method: MLE                                       
Start Date: 2016-10-24 09:20:00                         Log Likelihood: -191.0302                         
End Date: 2016-10-24 20:40:00                           AIC: 400.0603                                     
Number of observations: 34                              BIC: 413.7976                                     
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 -8.982     1.4673     -6.1216  0.0      (-11.8578 | -6.1061)     
AR(1)           

finish
模型的阶数为：(5, 1)
Normal ARIMA(5,1,1)                                                                                       
======================================================= ==================================================
Dependent Variable: Differenced diff                    Method: MLE                                       
Start Date: 2016-10-18 00:00:00                         Log Likelihood: -66.6869                          
End Date: 2016-10-18 11:40:00                           AIC: 149.3739                                     
Number of observations: 35                              BIC: 161.8167                                     
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 -0.1689    0.022      -7.6636  0.0      (-0.2121 | -0.1257)      
AR(1)           

finish
模型的阶数为：(3, 2)
Normal ARIMA(3,1,2)                                                                                       
======================================================= ==================================================
Dependent Variable: Differenced diff                    Method: MLE                                       
Start Date: 2016-10-18 11:20:00                         Log Likelihood: -28.2146                          
End Date: 2016-10-18 23:40:00                           AIC: 70.4292                                      
Number of observations: 37                              BIC: 81.7057                                      
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Constant                                 -0.0036    0.0014     -2.5686  0.0102   (-0.0064 | -0.0009)      
AR(1)           